In [1]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class QuantizeAndPruneLayer(nn.Module):
#     def __init__(self, k):
#         super(QuantizeAndPruneLayer, self).__init__()
#         self.k = k

#     def forward(self, x):
#         x = self.quantize_4bit(x)
#         x = x.float()
#         x = self.k_top_pruning(x, self.k)
#         return x

#     @staticmethod
#     def quantize_4bit(x):
#         min_val, max_val = x.min(), x.max()
#         x = (x - min_val) / (max_val - min_val)
#         x = (x * 15).round().int()
#         return x

#     @staticmethod
#     def k_top_pruning(x, k):
#         values, indices = torch.topk(x, k, dim=-1)
#         mask = torch.zeros_like(x).scatter_(-1, indices, 1)
#         return x * mask


In [2]:
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch

In [3]:
# from transformers import BertForSequenceClassification
# from transformers import TrainingArguments, Trainer
# import torch

# class BertForSequenceClassificationWithPruning(BertForSequenceClassification):
#     def __init__(self, config, k=10):
#         super(BertForSequenceClassificationWithPruning, self).__init__(config)
#         self.quantize_and_prune_layer = QuantizeAndPruneLayer(k)

#     def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
#         outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         sequence_output = outputs[1]
#         sequence_output = self.quantize_and_prune_layer(sequence_output)

#         logits = self.classifier(sequence_output)

#         probabilities = F.softmax(logits, dim=-1)

#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             return loss, probabilities
#         else:
#             return probabilities


In [4]:
# import math
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from transformers.models.bert.modeling_bert import BertAttention, BertSelfAttention, BertForSequenceClassification

# class QuantizeAndPruneLayer(nn.Module):
#     def __init__(self, k):
#         super(QuantizeAndPruneLayer, self).__init__()
#         self.k = k

#     def forward(self, x):
#         x = self.quantize_4bit(x)
#         x = self.softmax(x)
#         x = self.k_top_pruning(x, self.k)
#         return x

#     @staticmethod
#     def quantize_4bit(x):
#         x = x.int() >> 4  # Truncate the lower 4 bits
#         return x

#     @staticmethod
#     def softmax(x):
#         x = x.float()
#         return F.softmax(x, dim=-1)


#     @staticmethod
#     def k_top_pruning(x, k):
#         values, indices = torch.topk(x, k, dim=-1)
#         mask = torch.zeros_like(x).scatter_(-1, indices, 1)
#         return x * mask

# class QuantizedBertSelfAttention(BertSelfAttention):
#     def __init__(self, config, quantize_and_prune_layer):
#         super().__init__(config)
#         self.quantize_and_prune_layer = quantize_and_prune_layer

#     def forward(self, hidden_states, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=False):
#         mixed_query_layer = self.query(hidden_states)

#         # if this is a cross-attention layer and we have encoder states, apply the attention on them
#         if encoder_hidden_states is not None:
#             mixed_key_layer = self.key(encoder_hidden_states)
#             mixed_value_layer = self.value(encoder_hidden_states)
#             attention_mask = encoder_attention_mask
#         else:  # Self-attention
#             mixed_key_layer = self.key(hidden_states)
#             mixed_value_layer = self.value(hidden_states)

#         query_layer = self.transpose_for_scores(mixed_query_layer)
#         key_layer = self.transpose_for_scores(mixed_key_layer)

#         # Apply quantization and pruning to query and key layers
#         query_layer = self.quantize_and_prune_layer(query_layer)
#         key_layer = self.quantize_and_prune_layer(key_layer)

#         # Take the dot product between "query" and "key" to get the raw attention scores.
#         attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

#         attention_scores = attention_scores / math.sqrt(self.attention_head_size)
#         if attention_mask is not None:
#             # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
#             attention_scores = attention_scores + attention_mask

#         # Normalize the attention scores to probabilities.
#         attention_probs = nn.Softmax(dim=-1)(attention_scores)

#         # This is actually dropping out entire tokens to attend to, which might
#         # seem a bit unusual, but is taken from the original Transformer paper.
#         attention_probs = self.dropout(attention_probs)

#         # Mask heads if we want to
#         if head_mask is not None:
#             attention_probs = attention_probs * head_mask

#         value_layer = self.transpose_for_scores(mixed_value_layer)
#         context_layer = torch.matmul(attention_probs, value_layer)

#         context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
#         new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
#         context_layer = context_layer.view(*new_context_layer_shape)

#         outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

#         return outputs


# class QuantizedBertAttention(BertAttention):
#     def __init__(self, config, quantize_and_prune_layer):
#         super().__init__(config)
#         self.self = QuantizedBertSelfAttention(config, quantize_and_prune_layer)

# class BertForSequenceClassificationWithPruning(BertForSequenceClassification):
#     def __init__(self, config, k=10):
#         super(BertForSequenceClassificationWithPruning, self).__init__(config)
#         self.bert.encoder.layer[0].attention.self = QuantizedBertAttention(config, QuantizeAndPruneLayer(k))

#     def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
#         outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         sequence_output = outputs[1]

#         logits = self.classifier(sequence_output)

#         probabilities = F.softmax(logits, dim=-1)

#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             return loss, probabilities
#         else:
#             return probabilities


In [5]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.models.bert.modeling_bert import BertAttention, BertSelfAttention, BertForSequenceClassification
class MultiLevelBinaryQuantize(nn.Module):
    def __init__(self, levels):
        super(MultiLevelBinaryQuantize, self).__init__()
        self.levels = levels
        self.scale = nn.Parameter(torch.ones(1))

    def forward(self, x):
        x = x / self.scale
        x = torch.clamp(x, -self.levels, self.levels)
        x = torch.round(x)
        x = x * self.scale
        return x
class QuantizeAndPruneLayer(nn.Module):
    def __init__(self, k):
        super(QuantizeAndPruneLayer, self).__init__()
        self.k = k

    def forward(self, x):
        x = self.quantize_4bit(x)
        return x

    @staticmethod
    def quantize_4bit(x):
        x = x.int() >> 4  # Truncate the lower 4 bits
        return x

    @staticmethod
    def k_top_pruning(x, k):
        values, indices = torch.topk(x, k, dim=-1)
        mask = torch.zeros_like(x).scatter_(-1, indices, 1)
        return x * mask

class QuantizedBertSelfAttention(BertSelfAttention):
    def __init__(self, config, quantize_and_prune_layer):
        super().__init__(config)
        self.quantize_and_prune_layer = quantize_and_prune_layer

    def forward(self, hidden_states, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=False):
        mixed_query_layer = self.query(hidden_states)

        if encoder_hidden_states is not None:
            mixed_key_layer = self.key(encoder_hidden_states)
            mixed_value_layer = self.value(encoder_hidden_states)
            attention_mask = encoder_attention_mask
        else:  # Self-attention
            mixed_key_layer = self.key(hidden_states)
            mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)

        query_layer = self.quantize_and_prune_layer(query_layer).float()
        key_layer = self.quantize_and_prune_layer(key_layer).float()

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        attention_probs = self.quantize_and_prune_layer.k_top_pruning(attention_probs, self.quantize_and_prune_layer.k)

        attention_probs = self.dropout(attention_probs)

        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        value_layer = self.transpose_for_scores(mixed_value_layer)
        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        return outputs
class BertForSequenceClassificationWithPruning(BertForSequenceClassification):
    def __init__(self, config, k=10,levels=15):
        super(BertForSequenceClassificationWithPruning, self).__init__(config)
        self.bert.encoder.layer[0].attention.self = QuantizedBertSelfAttention(config, QuantizeAndPruneLayer(k))
        # self.quantize = MultiLevelBinaryQuantize(levels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[1]

        logits = self.classifier(sequence_output)
        # logits = self.quantize(logits)

        probabilities = F.softmax(logits, dim=-1)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss, probabilities
        else:
            return probabilities


In [6]:
!pip install datasets

In [7]:
from datasets import load_dataset
dataset = load_dataset("glue","mrpc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [9]:
!pip install transformers[torch]

In [10]:
!pip install accelerate -U

In [11]:
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import glue_convert_examples_to_features
from transformers import GlueDataset, GlueDataTrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# data_args = GlueDataTrainingArguments(task_name="mrpc", data_dir="./mrpc")

mapped_dataset = dataset.map(lambda x: tokenizer(x["sentence1"],x["sentence2"]),batched=True)
# train_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="train")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:

model = BertForSequenceClassificationWithPruning.from_pretrained('bert-base-uncased', k=10)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mapped_dataset["train"],
    eval_dataset=mapped_dataset["validation"],
    data_collator=data_collator
)

trainer.train()


Some weights of BertForSequenceClassificationWithPruning were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.578100
1000,0.411200


TrainOutput(global_step=1377, training_loss=0.4361565506103338, metrics={'train_runtime': 205.6539, 'train_samples_per_second': 53.507, 'train_steps_per_second': 6.696, 'total_flos': 405324636337200.0, 'train_loss': 0.4361565506103338, 'epoch': 3.0})

In [14]:
preds = trainer.predict(mapped_dataset["test"])

In [15]:
import numpy
logits, labels, _ = preds
predictions = numpy.argmax(logits, axis=-1)

In [16]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [17]:
import evaluate

metric = evaluate.load("glue","mrpc")

In [18]:
metric.compute(predictions=predictions,references=labels)

{'accuracy': 0.8330434782608696, 'f1': 0.8802992518703243}

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import torch.nn.init as init
import copy
import math
from transformers.models.bert.modeling_bert import BertAttention, BertSelfAttention, BertForSequenceClassification
class MultiLevelBinaryQuantize(nn.Module):
    def __init__(self, levels):
        super(MultiLevelBinaryQuantize, self).__init__()
        self.levels = levels
        self.scale = nn.Parameter(torch.ones(1))

    def forward(self, x):
        x = x / self.scale
        x = torch.clamp(x, -self.levels, self.levels)
        x = torch.round(x)
        x = x * self.scale
        return x
class QuantizeAndPruneLayer(nn.Module):
    def __init__(self, k):
        super(QuantizeAndPruneLayer, self).__init__()
        self.k = k

    def forward(self, x):
        x = self.quantize_4bit(x)
        return x

    @staticmethod
    def quantize_4bit(x):
        x = x.int() >> 4  # Truncate the lower 4 bits
        return x

    @staticmethod
    def k_top_pruning(x, k):
        values, indices = torch.topk(x, k, dim=-1)
        mask = torch.zeros_like(x).scatter_(-1, indices, 1)
        return x * mask

class QuantizedBertSelfAttention(BertSelfAttention):
    def __init__(self, config, quantize_and_prune_layer):
        super().__init__(config)
        self.quantize_and_prune_layer = quantize_and_prune_layer

    def forward(self, hidden_states, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=False):
        mixed_query_layer = self.query(hidden_states)

        if encoder_hidden_states is not None:
            mixed_key_layer = self.key(encoder_hidden_states)
            mixed_value_layer = self.value(encoder_hidden_states)
            attention_mask = encoder_attention_mask
        else:  # Self-attention
            mixed_key_layer = self.key(hidden_states)
            mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)

        query_layer = self.quantize_and_prune_layer(query_layer).float()
        key_layer = self.quantize_and_prune_layer(key_layer).float()

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        attention_probs = self.quantize_and_prune_layer.k_top_pruning(attention_probs, self.quantize_and_prune_layer.k)

        attention_probs = self.dropout(attention_probs)

        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        value_layer = self.transpose_for_scores(mixed_value_layer)
        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        return outputs
class BertForSequenceClassificationWithPruning(BertForSequenceClassification):
    def __init__(self, config, k=10,levels=15):
        super(BertForSequenceClassificationWithPruning, self).__init__(config)
        self.bert.encoder.layer[0].attention.self = QuantizedBertSelfAttention(config, QuantizeAndPruneLayer(k))
        # self.quantize = MultiLevelBinaryQuantize(levels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[1]

        logits = self.classifier(sequence_output)
        # logits = self.quantize(logits)

        probabilities = F.softmax(logits, dim=-1)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss, probabilities
        else:
            return probabilities
def gamma_initializer(shape, gain= .5, dtype=None):
    return gain * torch.arange(shape,0,-1).float()/shape

class Round(Function):
    @staticmethod
    def forward(ctx, input):
        return torch.floor(input + 0.5)

    @staticmethod
    def backward(ctx, grad_output):
        # Gradient is passed through unchanged
        return grad_output

class HighestPowerOf2(nn.Module):
    def __init__(self):
        super(HighestPowerOf2, self).__init__()

    def forward(self, n):
        n = n.clone().detach().requires_grad_(True)

        # Calculate the power
        p = torch.log2(n)

        # Use custom round operation
        p = Round.apply(p)
        return 2 ** p

class FPQuantize(nn.Module):
    def __init__(self, w, f):
        super(FPQuantize, self).__init__()
        self.w = w
        self.f = f
        self.i = w - f
        self.max_val = float(2 ** (self.i - 1) - 2 ** (-f))
        self.min_val = float(-2 ** (self.i - 1))
        self.n = float(2 ** f)

    def forward(self, x):
        x = Round.apply(x * self.n + 0.5) / self.n
        x = torch.clamp(x, min=self.min_val, max=self.max_val)
        return x


class Abs(Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x.abs()

    @staticmethod
    def backward(ctx, grad_output):
        x, = ctx.saved_variables
        return grad_output * x.sign()

class Binarize(Function):
    @staticmethod
    def forward(ctx, x):
        ctx._mask = (x.ge(-1) * x.le(1))
        clipped = torch.clamp(x, -1, 1)
        rounded = torch.sign(clipped).clone().detach()
        sp = (rounded - clipped.clone().detach()).detach()
        sp.requires_grad = False
        return clipped + sp

    @staticmethod
    def backward(ctx, grad_output):
        mask = torch.autograd.Variable(ctx._mask.type_as(grad_output.data))
        return grad_output * mask

class FunRes(Function):
    @staticmethod
    def forward(ctx, x, out_bin, out):
        o_bin = out_bin + out
        resid = x - out
        return o_bin, resid

    @staticmethod
    def backward(ctx, grad_o_bin, grad_resid):
        # Initialize gradients for x, out_bin, and out
        grad_x = grad_out_bin = grad_out = None

        if ctx.needs_input_grad[0]:
            # Compute gradient with respect to x
            grad_x = grad_resid
        if ctx.needs_input_grad[1]:
            # Compute gradient with respect to out_bin
            grad_out_bin = grad_o_bin
        if ctx.needs_input_grad[2]:
            # Compute gradient with respect to out
            grad_out = grad_o_bin

        # Return the gradients, each with respect to its corresponding input
        return grad_x, grad_out_bin, grad_out

class MLBinarize(nn.Module):
    def __init__(self, levels, gamma):
        super(MLBinarize, self).__init__()
        self.levels = levels
        self.gamma = gamma

        self.highest_power_of_2 = HighestPowerOf2()

    def forward(self, x):
        resid = x
        out_bin = 0
        for l in range(self.levels):
            out = Binarize.apply(resid) * self.highest_power_of_2(Abs.apply(self.gamma[l]))
            out_bin, resid = FunRes.apply(resid, out_bin, out)
        return out_bin

class ResidualSign(nn.Module):
    def __init__(self, levels=1):
        super(ResidualSign, self).__init__()
        self.levels = levels
        ars = np.arange(self.levels) + 1.0
        ars = ars[::-1]
        means = ars / np.sum(ars)
        self.means = nn.Parameter(torch.tensor(means, dtype=torch.float32))

        self.param = {
            'levels': self.levels
        }
        self.type = 'res'

    def forward(self, x):
        if self.means is None:
            raise ValueError("Call set_means(X) before using the layer.")

        resid = x
        out_bin = 0
        for l in range(self.levels):
            out = Binarize.apply(resid) * Abs.apply(self.means[l])
            out_bin, resid = FunRes.apply(resid, out_bin, out)

        return out_bin

    def _save_to_state_dict(self, destination=None, prefix='', keep_vars=False):
        # Save additional attributes along with the state
        destination[prefix + 'param'] = self.param
        destination[prefix + 'type'] = self.type
        super()._save_to_state_dict(destination, prefix, keep_vars)

    def _load_from_state_dict(self, state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs):
        self.param = state_dict.pop(prefix + 'param', None)
        self.type = state_dict.pop(prefix + 'type', None)
        super()._load_from_state_dict(state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs)

    def set_means(self, X):
        means = np.zeros((self.levels))
        means[0] = 1
        resid = np.clip(X, -1, 1)
        approx = 0
        for l in range(self.levels):
            m = np.mean(np.absolute(resid))
            out = np.sign(resid) * m
            approx = approx + out
            resid = resid - out
            means[l] = m
            err = np.mean((approx - np.clip(X, -1, 1))**2)

        means = means / np.sum(means)

class BinaryInput(nn.Module):
    def __init__(self, levels=2):
        super(BinaryInput, self).__init__()
        self.levels = levels

        ars = np.arange(self.levels) + 1.0
        ars = ars[::-1]
        means = ars / np.sum(ars)
        self.means = nn.Parameter(torch.tensor(means, dtype=torch.float32), requires_grad=True)

        self.param = {
            'levels': self.levels
        }
        self.type = 'input'

    def forward(self, x):
        bn_module = MLBinarize(self.levels, self.means)
        return bn_module(x)

    def _save_to_state_dict(self, destination=None, prefix='', keep_vars=False):
        # Save additional attributes along with the state
        destination[prefix + 'param'] = self.param
        destination[prefix + 'type'] = self.type
        super()._save_to_state_dict(destination, prefix, keep_vars)

    def _load_from_state_dict(self, state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs):
        self.param = state_dict.pop(prefix + 'param', None)
        self.type = state_dict.pop(prefix + 'type', None)
        super()._load_from_state_dict(state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs)

class FunBinaryWeight(torch.autograd.Function):
    @staticmethod
    def forward(ctx, weight, gamma):
        ctx.save_for_backward(weight, gamma)
        clamped_w = gamma * weight
        return clamped_w

    @staticmethod
    def backward(ctx, grad_output):
        weight, gamma = ctx.saved_tensors

        grad_weight = grad_output * gamma  # Gradient with respect to weight
        grad_gamma = torch.sum(grad_output * weight)  # Gradient with respect to gamma

        return grad_weight, grad_gamma

class BinaryLinear(nn.Linear):
    def __init__(self, n_in, n_out, w_bits=2, a_bits=2):
        super(BinaryLinear, self).__init__(n_in, n_out, bias=False)
        self.n_in = n_in
        self.n_out = n_out
        self.w_levels = w_bits
        self.a_levels = a_bits

        stdv = 1 / np.sqrt(self.n_in)
        w = np.random.normal(loc=0.0, scale=stdv, size=(self.n_in, self.n_out)).astype(np.float32)
        self.weight = nn.Parameter(torch.tensor(w))
        self.gamma = nn.Parameter(gamma_initializer(self.w_levels, 0.5), requires_grad=True)

        self.mlb = MLBinarize(self.w_levels, self.gamma)
        self.amlb = ResidualSign(a_bits)

    def set_weight(self, weight):
        self.weight = nn.Parameter(torch.tensor(weight))

    def forward(self, x):
        if self.a_levels > 0:
            x = self.amlb(x)
        clamped_w = self.mlb(self.weight)
        out = F.linear(x, clamped_w, None)
        return out


class BinaryConv2d(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, stride=1,
                groups=1, dilation=1, padding_mode='zeros', levels=1):

        super(BinaryConv2d, self).__init__(in_channels, out_channels, kernel_size, stride, padding,
                                    dilation=dilation, groups=groups, padding_mode=padding_mode, bias=False)

        self.w_levels = levels

        if type(kernel_size) is int:
            self.k = (kernel_size, kernel_size)
        else:
            self.k = kernel_size

        stdv = 1 / np.sqrt(self.k[0] * self.k[1] * self.in_channels)
        w = np.random.normal(loc=0.0, scale=stdv, size=(self.out_channels, self.in_channels, self.k[0], self.k[1])).astype(np.float32)
        self.weight = nn.Parameter(torch.tensor(w))
        self.gamma = nn.Parameter(gamma_initializer(self.w_levels, 0.5), requires_grad=True)

        self.mlb = MLBinarize(self.w_levels, self.gamma)

        self.param = {
            self.w_levels
        }
        self.type = 'conv2d'

    def set_weight(self, weight):
        self.weight = nn.Parameter(torch.tensor(weight))

    def forward(self, x):
        clamped_w = self.mlb(self.weight)
        out = F.conv2d(x, clamped_w, padding=self.padding, stride=self.stride, dilation=self.dilation, groups=self.groups)
        return out

    def _save_to_state_dict(self, destination=None, prefix='', keep_vars=False):
        # Save additional attributes along with the state
        destination[prefix + 'param'] = self.param
        destination[prefix + 'type'] = self.type
        super()._save_to_state_dict(destination, prefix, keep_vars)

    def _load_from_state_dict(self, state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs):
        self.param = state_dict.pop(prefix + 'param', None)
        self.type = state_dict.pop(prefix + 'type', None)
        super()._load_from_state_dict(state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs)

def add_quant_op(module, layer_counter, a_bits=8, w_bits=8):
    for name, child in module.named_children():
        if isinstance(child, nn.Linear):
            layer_counter[0] += 1
            quant_linear = BinaryLinear(child.in_features, child.out_features,
                                        a_bits=a_bits, w_bits=w_bits)

            quant_linear.weight.data = child.weight
            module._modules[name] = quant_linear
        else:
            add_quant_op(child, layer_counter, a_bits=a_bits, w_bits=w_bits)


def prepare(model, inplace=False, a_bits=8, w_bits=8):
    if not inplace:
        model = copy.deepcopy(model)
    layer_counter = [0]
    add_quant_op(model, layer_counter, a_bits=a_bits, w_bits=w_bits)
    return model
model = BertForSequenceClassificationWithPruning.from_pretrained('bert-base-uncased', k=10)
model = prepare(model)

Some weights of BertForSequenceClassificationWithPruning were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mapped_dataset["train"],
    eval_dataset=mapped_dataset["validation"],
    data_collator=data_collator
)

trainer.train()

<ipython-input-19-3f5d5a4c55db>:161: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  x, = ctx.saved_variables


Step,Training Loss
100,0.647800
200,0.639800
300,0.634700
400,0.653300


AttributeError: 'dict' object has no attribute 'device'

In [42]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import torch.nn.init as init
import copy
import math
from transformers.models.bert.modeling_bert import BertAttention, BertSelfAttention, BertForSequenceClassification
class MultiLevelBinaryQuantize(nn.Module):
    def __init__(self, levels):
        super(MultiLevelBinaryQuantize, self).__init__()
        self.levels = levels
        self.scale = nn.Parameter(torch.ones(1))

    def forward(self, x):
        x = x / self.scale
        x = torch.clamp(x, -self.levels, self.levels)
        x = torch.round(x)
        x = x * self.scale
        return x
class QuantizeAndPruneLayer(nn.Module):
    def __init__(self, k):
        super(QuantizeAndPruneLayer, self).__init__()
        self.k = k

    def forward(self, x):
        x = self.quantize_4bit(x)
        return x

    @staticmethod
    def quantize_4bit(x):
        x = x.int() >> 64  # Truncate the lower 4 bits
        return x

    @staticmethod
    def k_top_pruning(x, k):
        values, indices = torch.topk(x, k, dim=-1)
        mask = torch.zeros_like(x).scatter_(-1, indices, 1)
        return x * mask

class QuantizedBertSelfAttention(BertSelfAttention):
    def __init__(self, config, quantize_and_prune_layer):
        super().__init__(config)
        self.quantize_and_prune_layer = quantize_and_prune_layer

    def forward(self, hidden_states, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=False):
        mixed_query_layer = self.query(hidden_states)

        if encoder_hidden_states is not None:
            mixed_key_layer = self.key(encoder_hidden_states)
            mixed_value_layer = self.value(encoder_hidden_states)
            attention_mask = encoder_attention_mask
        else:  # Self-attention
            mixed_key_layer = self.key(hidden_states)
            mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)

        query_layer = self.quantize_and_prune_layer(query_layer).float()
        key_layer = self.quantize_and_prune_layer(key_layer).float()

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        attention_probs = self.quantize_and_prune_layer.k_top_pruning(attention_probs, self.quantize_and_prune_layer.k)

        attention_probs = self.dropout(attention_probs)

        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        value_layer = self.transpose_for_scores(mixed_value_layer)
        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        return outputs
class BertForSequenceClassificationWithPruning(BertForSequenceClassification):
    def __init__(self, config, k=10,levels=15):
        super(BertForSequenceClassificationWithPruning, self).__init__(config)
        self.bert.encoder.layer[0].attention.self = QuantizedBertSelfAttention(config, QuantizeAndPruneLayer(k))
        # self.quantize = MultiLevelBinaryQuantize(levels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[1]

        logits = self.classifier(sequence_output)
        # logits = self.quantize(logits)

        probabilities = F.softmax(logits, dim=-1)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss, probabilities
        else:
            return probabilities
class Round(Function):
    @staticmethod
    def forward(self, input):
        sign = torch.sign(input)
        output = sign * torch.floor(torch.abs(input) + 0.5)
        return output

    @staticmethod
    def backward(self, grad_output):
        grad_input = grad_output.clone()
        return grad_input

class FunLSQ(Function):
    @staticmethod
    def forward(ctx, weight, alpha, g, Qn, Qp, per_channel=False):
        #LEARNED STEP SIZE QUANTIZATION
        ctx.save_for_backward(weight, alpha)
        ctx.other = g, Qn, Qp, per_channel
        if per_channel:
            sizes = weight.size()
            weight = weight.contiguous().view(weight.size()[0], -1)
            weight = torch.transpose(weight, 0, 1)
            alpha = torch.broadcast_to(alpha, weight.size())
            w_q = Round.apply(torch.div(weight, alpha).clamp(Qn, Qp))
            w_q = w_q * alpha
            w_q = torch.transpose(w_q, 0, 1)
            w_q = w_q.contiguous().view(sizes)
        else:
            w_q = Round.apply(torch.div(weight, alpha).clamp(Qn, Qp))
            w_q = w_q * alpha
        return w_q

    @staticmethod
    def backward(ctx, grad_weight):
        #LEARNED STEP SIZE QUANTIZATION
        weight, alpha = ctx.saved_tensors
        g, Qn, Qp, per_channel = ctx.other
        if per_channel:
            sizes = weight.size()
            weight = weight.contiguous().view(weight.size()[0], -1)
            weight = torch.transpose(weight, 0, 1)
            alpha = torch.broadcast_to(alpha, weight.size())
            q_w = weight / alpha
            q_w = torch.transpose(q_w, 0, 1)
            q_w = q_w.contiguous().view(sizes)
        else:
            q_w = weight / alpha
        smaller = (q_w < Qn).float()
        bigger = (q_w > Qp).float()
        between = 1.0 - smaller -bigger
        if per_channel:
            grad_alpha = ((smaller * Qn + bigger * Qp +
                between * Round.apply(q_w) - between * q_w)*grad_weight * g)
            grad_alpha = grad_alpha.contiguous().view(grad_alpha.size()[0], -1).sum(dim=1)
        else:
            grad_alpha = ((smaller * Qn + bigger * Qp +
                between * Round.apply(q_w) - between * q_w)*grad_weight * g).sum().unsqueeze(dim=0) #?
        grad_weight = between * grad_weight
        return grad_weight, grad_alpha, None, None, None, None

def grad_scale(x, scale):
    y = x
    y_grad = x * scale
    return (y - y_grad).detach() + y_grad

def round_pass(x):
    y = x.round()
    y_grad = x
    return (y - y_grad).detach() + y_grad

class LSQActivationQuantizer(nn.Module):
    def __init__(self, a_bits, all_positive=False, batch_init = 20):
        #activations per-channel
        super(LSQActivationQuantizer, self).__init__()
        self.a_bits = a_bits
        self.all_positive = all_positive
        self.batch_init = batch_init
        if self.all_positive:
            # unsigned activation is quantized to [0, 2^b-1]
            self.Qn = 0
            self.Qp = 2 ** self.a_bits - 1
        else:
            # signed weight/activation is quantized to [-2^(b-1), 2^(b-1)-1]
            self.Qn = - 2 ** (self.a_bits - 1)
            self.Qp = 2 ** (self.a_bits - 1) - 1
        self.s = torch.nn.Parameter(torch.ones(1), requires_grad=True)  #V2
        self.init_state = 0

    def forward(self, activation):
        if self.a_bits == 32:
            q_a = activation
        elif self.a_bits == 1:
            print('！Binary quantization is not supported ！')
            assert self.a_bits != 1
        else:
            if self.init_state==0:
                self.g = 1.0/math.sqrt(activation.numel() * self.Qp)
                self.init_state += 1
            q_a = FunLSQ.apply(activation, self.s, self.g, self.Qn, self.Qp)

        return q_a

class LSQWeightQuantizer(nn.Module):
    def __init__(self, w_bits, all_positive=False, per_channel=False, batch_init = 20):
        super(LSQWeightQuantizer, self).__init__()
        self.w_bits = w_bits
        self.all_positive = all_positive
        self.batch_init = batch_init
        if self.all_positive:
            # unsigned activation is quantized to [0, 2^b-1]
            self.Qn = 0
            self.Qp = 2 ** w_bits - 1
        else:
            # signed weight/activation is quantized to [-2^(b-1), 2^(b-1)-1]
            self.Qn = - 2 ** (w_bits - 1)
            self.Qp = 2 ** (w_bits - 1) - 1
        self.per_channel = per_channel
        self.s = torch.nn.Parameter(torch.ones(1), requires_grad=True)
        self.init_state = 0

    def forward(self, weight):
        if self.init_state==0:
            self.g = 1.0/math.sqrt(weight.numel() * self.Qp)
            if self.per_channel:
                weight_tmp = weight.detach().contiguous().view(weight.size()[0], -1)
                self.s.data = torch.mean(torch.abs(weight_tmp), dim=1)*2/(math.sqrt(self.Qp))
            else:
                self.s.data = torch.mean(torch.abs(weight.detach()))*2/(math.sqrt(self.Qp))
            self.init_state += 1
        elif self.init_state<self.batch_init:
            if self.per_channel:
                weight_tmp = weight.detach().contiguous().view(weight.size()[0], -1)
                self.s.data = 0.9*self.s.data + 0.1*torch.mean(torch.abs(weight_tmp), dim=1)*2/(math.sqrt(self.Qp))
            else:
                self.s.data = 0.9*self.s.data + 0.1*torch.mean(torch.abs(weight.detach()))*2/(math.sqrt(self.Qp))
            self.init_state += 1
        elif self.init_state==self.batch_init:
            self.init_state += 1
        if self.w_bits == 32:
            w_q = weight
        elif self.w_bits == 1:
            print('！Binary quantization is not supported ！')
            assert self.w_bits != 1
        else:
            w_q = FunLSQ.apply(weight, self.s, self.g, self.Qn, self.Qp, self.per_channel)

        return w_q

class QuantConv2d(nn.Conv2d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=0,
                 dilation=1,
                 groups=1,
                 bias=True,
                 padding_mode='zeros',
                 a_bits=8,
                 w_bits=8,
                 quant_inference=False,
                 all_positive=False,
                 per_channel=False,
                 batch_init = 20):
        super(QuantConv2d, self).__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups,
                                          bias, padding_mode)
        self.quant_inference = quant_inference
        self.activation_quantizer = LSQActivationQuantizer(a_bits=a_bits, all_positive=all_positive,batch_init = batch_init)
        self.weight_quantizer = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=per_channel,batch_init = batch_init)

    def forward(self, input):
        quant_input = self.activation_quantizer(input)
        if not self.quant_inference:
            quant_weight = self.weight_quantizer(self.weight)
        else:
            quant_weight = self.weight

        output = F.conv2d(quant_input, quant_weight, self.bias, self.stride, self.padding, self.dilation,
                          self.groups)
        return output


class QuantConvTranspose2d(nn.ConvTranspose2d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=0,
                 output_padding=0,
                 dilation=1,
                 groups=1,
                 bias=True,
                 padding_mode='zeros',
                 a_bits=8,
                 w_bits=8,
                 quant_inference=False,
                 all_positive=False,
                 per_channel=False,
                 batch_init = 20):
        super(QuantConvTranspose2d, self).__init__(in_channels, out_channels, kernel_size, stride, padding, output_padding,
                                                   dilation, groups, bias, padding_mode)
        self.quant_inference = quant_inference
        self.activation_quantizer = LSQActivationQuantizer(a_bits=a_bits, all_positive=all_positive,batch_init = batch_init)
        self.weight_quantizer = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=per_channel,batch_init = batch_init)

    def forward(self, input):
        quant_input = self.activation_quantizer(input)
        if not self.quant_inference:
            quant_weight = self.weight_quantizer(self.weight)
        else:
            quant_weight = self.weight
        output = F.conv_transpose2d(quant_input, quant_weight, self.bias, self.stride, self.padding, self.output_padding,
                                    self.groups, self.dilation)
        return output


class QuantLinear(nn.Linear):
    def __init__(self,
                 in_features,
                 out_features,
                 bias=True,
                 a_bits=8,
                 w_bits=8,
                 quant_inference=False,
                 all_positive=False,
                 per_channel=False,
                 batch_init = 20):
        super(QuantLinear, self).__init__(in_features, out_features, bias)
        self.quant_inference = quant_inference
        self.activation_quantizer = LSQActivationQuantizer(a_bits=a_bits, all_positive=all_positive,batch_init = batch_init)
        self.weight_quantizer = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=per_channel,batch_init = batch_init)

    def forward(self, input):
        quant_input = self.activation_quantizer(input)
        if not self.quant_inference:
            quant_weight = self.weight_quantizer(self.weight)
        else:
            quant_weight = self.weight
        output = F.linear(quant_input, quant_weight, self.bias)
        return output

class QuantBatchNorm2d(nn.BatchNorm2d):
    def __init__(self, num_features: int, eps: float = 0.00001, momentum: float = 0.1, affine: bool = True, track_running_stats: bool = True, device=None, dtype=None, w_bits=8, all_positive=False, batch_init=20) -> None:
        super(QuantBatchNorm2d, self).__init__(num_features, eps, momentum, affine, track_running_stats, device, dtype)

        self.w_bits = w_bits

        self.quant_w = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=False, batch_init = batch_init)
        self.quant_b = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=False, batch_init = batch_init)

    def forward(self, x):
        if self.weight is not None:
            self.weight.data = self.quant_w(self.weight)
        if self.bias is not None:
            self.bias.data = self.quant_b(self.bias)

        return super(QuantBatchNorm2d, self).forward(x)

    def state_dict(self, destination=None, prefix='', keep_vars=False):
        # Save additional attributes along with the state
        state = super(QuantBatchNorm2d, self).state_dict(destination, prefix, keep_vars)
        state[prefix + 'param'] = {
            'w_bits': self.w_bits
        }
        state[prefix + 'type'] = 'batchnorm2d'
        return state

class QuantBatchNorm1d(nn.BatchNorm1d):
    def __init__(self, num_features: int, eps: float = 0.00001, momentum: float = 0.1, affine: bool = True, track_running_stats: bool = True, device=None, dtype=None, w_bits=8, all_positive=False, batch_init=20) -> None:
        super(QuantBatchNorm1d, self).__init__(num_features, eps, momentum, affine, track_running_stats, device, dtype)

        self.w_bits = w_bits

        self.quant_w = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=False, batch_init = batch_init)
        self.quant_b = LSQWeightQuantizer(w_bits=w_bits, all_positive=all_positive, per_channel=False, batch_init = batch_init)

    def forward(self, x):
        if self.weight is not None:
            self.weight.data = self.quant_w(self.weight)
        if self.bias is not None:
            self.bias.data = self.quant_b(self.bias)

        return super(QuantBatchNorm1d, self).forward(x)

    def state_dict(self, destination=None, prefix='', keep_vars=False):
        # Save additional attributes along with the state
        state = super(QuantBatchNorm1d, self).state_dict(destination, prefix, keep_vars)
        state[prefix + 'param'] = {
            'w_bits': self.w_bits
        }
        state[prefix + 'type'] = 'batchnorm1d'
        return state

def add_quant_op(module, layer_counter, a_bits=8, w_bits=8,
                 quant_inference=False, all_positive=False, per_channel=False,
                 batch_init = 20, length=1):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d):
            layer_counter[0] += 1
            if layer_counter[0] > 1:
                if child.bias is not None:
                    quant_conv = QuantConv2d(child.in_channels, child.out_channels,
                                             child.kernel_size, stride=child.stride,
                                             padding=child.padding, dilation=child.dilation,
                                             groups=child.groups, bias=True, padding_mode=child.padding_mode,
                                             a_bits=a_bits, w_bits=w_bits, quant_inference=quant_inference,
                                             all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)
                    quant_conv.bias.data = child.bias
                else:
                    quant_conv = QuantConv2d(child.in_channels, child.out_channels,
                                             child.kernel_size, stride=child.stride,
                                             padding=child.padding, dilation=child.dilation,
                                             groups=child.groups, bias=False, padding_mode=child.padding_mode,
                                             a_bits=a_bits, w_bits=w_bits, quant_inference=quant_inference,
                                             all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)
                quant_conv.weight.data = child.weight
                module._modules[name] = quant_conv
        elif isinstance(child, nn.ConvTranspose2d):
            layer_counter[0] += 1
            if layer_counter[0] > 1:
                if child.bias is not None:
                    quant_conv_transpose = QuantConvTranspose2d(child.in_channels,
                                                                child.out_channels,
                                                                child.kernel_size,
                                                                stride=child.stride,
                                                                padding=child.padding,
                                                                output_padding=child.output_padding,
                                                                dilation=child.dilation,
                                                                groups=child.groups,
                                                                bias=True,
                                                                padding_mode=child.padding_mode,
                                                                a_bits=a_bits,
                                                                w_bits=w_bits,
                                                                quant_inference=quant_inference,
                                             all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)
                    quant_conv_transpose.bias.data = child.bias
                else:
                    quant_conv_transpose = QuantConvTranspose2d(child.in_channels,
                                                                child.out_channels,
                                                                child.kernel_size,
                                                                stride=child.stride,
                                                                padding=child.padding,
                                                                output_padding=child.output_padding,
                                                                dilation=child.dilation,
                                                                groups=child.groups, bias=False,
                                                                padding_mode=child.padding_mode,
                                                                a_bits=a_bits,
                                                                w_bits=w_bits,
                                                                quant_inference=quant_inference,
                                             all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)
                quant_conv_transpose.weight.data = child.weight
                module._modules[name] = quant_conv_transpose
        elif isinstance(child, nn.Linear):
            layer_counter[0] += 1
            if layer_counter[0] > 1 and layer_counter[0] < length:
                if child.bias is not None:
                    quant_linear = QuantLinear(child.in_features, child.out_features,
                                               bias=True, a_bits=a_bits, w_bits=w_bits,
                                               quant_inference=quant_inference,
                                             all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)
                    quant_linear.bias.data = child.bias
                else:
                    quant_linear = QuantLinear(child.in_features, child.out_features,
                                               bias=False, a_bits=a_bits, w_bits=w_bits,
                                               quant_inference=quant_inference,
                                             all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)
                quant_linear.weight.data = child.weight
                module._modules[name] = quant_linear
        else:
            add_quant_op(child, layer_counter, a_bits=a_bits, w_bits=w_bits,
                         quant_inference=quant_inference, all_positive=all_positive, per_channel=per_channel, batch_init = batch_init)


def prepare(model, inplace=False, a_bits=8, w_bits=8, quant_inference=False,
            all_positive=False, per_channel=False, batch_init = 20):
    if not inplace:
        model = copy.deepcopy(model)
    layer_counter = [0]
    l = len(list(model.named_children()))
    add_quant_op(model, layer_counter, a_bits=a_bits, w_bits=w_bits,
                 quant_inference=quant_inference, all_positive=all_positive,
                 per_channel=per_channel, batch_init = batch_init, length=l)
    return model
model = BertForSequenceClassificationWithPruning.from_pretrained('bert-base-uncased', k=10)
model = prepare(model)

Some weights of BertForSequenceClassificationWithPruning were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mapped_dataset["train"],
    eval_dataset=mapped_dataset["validation"],
    data_collator=data_collator
)

trainer.train()

Step,Training Loss
100,0.650400
200,0.594400
300,0.579700
400,0.531100
500,0.420200
600,0.336000
700,0.371300
800,0.392800
900,0.325800
1000,0.135300


TrainOutput(global_step=1377, training_loss=0.35207031599057265, metrics={'train_runtime': 194.8468, 'train_samples_per_second': 56.475, 'train_steps_per_second': 7.067, 'total_flos': 405324636337200.0, 'train_loss': 0.35207031599057265, 'epoch': 3.0})

In [41]:
import numpy
import evaluate

metric = evaluate.load("glue","mrpc")
preds = trainer.predict(mapped_dataset["test"])
logits, labels, _ = preds
predictions = numpy.argmax(logits, axis=-1)
metric.compute(predictions=predictions,references=labels)

{'accuracy': 0.7942028985507247, 'f1': 0.850147741663149}